# Lab Session \# 07


---


by Josué Obregón <br>
BDA712-00 - Machine Learning Programming <br>
Department of Big Data Analytics - Kyung Hee University<br>

## Objective

The objective of this session is learn how to implement a one layer neural network.

# Getting the data

In [1]:
import gdown

In [2]:
!mkdir data

In [3]:
urls = ['https://drive.google.com/uc?export=download&id=1MnUH3W1Jm8LVBqEJ1M0m5l9_Q8hJZqrz', # train-labels-idx1-ubyte.gz  https://drive.google.com/file/d/1MnUH3W1Jm8LVBqEJ1M0m5l9_Q8hJZqrz/view?usp=sharing       
        'https://drive.google.com/uc?export=download&id=1AZLWnMx1xe3vN1naEswKL19I02YrA7_J', # train-images-idx3-ubyte.gz  https://drive.google.com/file/d/1AZLWnMx1xe3vN1naEswKL19I02YrA7_J/view?usp=sharing       
        'https://drive.google.com/uc?export=download&id=1Hw8QHRxmI4w-ZAo5yzVjDB3UnUPAVv4u', # t10k-labels-idx1-ubyte.gz  https://drive.google.com/file/d/1Hw8QHRxmI4w-ZAo5yzVjDB3UnUPAVv4u/view?usp=sharing       
        'https://drive.google.com/uc?export=download&id=1EHdJfVQs1ZiRhCoEldMc9lTJ-5Nz5GaV', # t10k-images-idx3-ubyte.gz  https://drive.google.com/file/d/1EHdJfVQs1ZiRhCoEldMc9lTJ-5Nz5GaV/view?usp=sharing       
      ]
outputs = ['train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
           't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1MnUH3W1Jm8LVBqEJ1M0m5l9_Q8hJZqrz
To: /content/data/train-labels-idx1-ubyte.gz
100%|██████████| 28.9k/28.9k [00:00<00:00, 26.5MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1AZLWnMx1xe3vN1naEswKL19I02YrA7_J
To: /content/data/train-images-idx3-ubyte.gz
100%|██████████| 9.91M/9.91M [00:00<00:00, 49.1MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1Hw8QHRxmI4w-ZAo5yzVjDB3UnUPAVv4u
To: /content/data/t10k-labels-idx1-ubyte.gz
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.46MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1EHdJfVQs1ZiRhCoEldMc9lTJ-5Nz5GaV
To: /content/data/t10k-images-idx3-ubyte.gz
100%|██████████| 1.65M/1.65M [00:00<00:00, 139MB/s]


# Preliminaries

Let's import the data and prepare the variables that we will need for our laboratory

In [4]:
import numpy as np
import gzip
import struct
import matplotlib.pyplot as plt
import seaborn as sns

Functions for loading images and labels of MNIST dataset

In [5]:
import numpy as np
import gzip
import struct


def load_images(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Read the header information into a bunch of variables
        _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
        # Read all the pixels into a NumPy array of bytes:
        all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
        # Reshape the pixels into a matrix where each line is an image:
        return all_pixels.reshape(n_images, columns * rows)

def load_labels(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Skip the header bytes:
        f.read(8)
        # Read all the labels into a list:
        all_labels = f.read()
        # Reshape the list of labels into a one-column matrix:
        return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)

Let's create our `one_hot_encode` and `prepend_bias` function

In [6]:
def one_hot_encode(Y):
  unique_labels =  np.unique(Y)
  return (Y == unique_labels).astype(int)

In [7]:
def prepend_bias(X):
    # Insert a column of 1s in the position 0 of X.
    # (“axis=1” stands for: “insert a column, not a row”)
  return np.insert(X, 0, 1, axis=1) # insert(arr, obj, values, axis=None)

Let's load our data into our main variables

In [8]:
# 60000 images, each 784 elements (28 * 28 pixels)
X_train = load_images("data/train-images-idx3-ubyte.gz")

# 10000 images, each 784 elements, with the same structure as X_train
X_test = load_images("data/t10k-images-idx3-ubyte.gz")

# 60K labels, each a single digit from 0 to 9
Y_train_unencoded = load_labels("data/train-labels-idx1-ubyte.gz")

# 60K labels, each consisting of 10 one-hot encoded elements
Y_train = one_hot_encode(Y_train_unencoded)

# 10000 labels, each a single digit from 0 to 9
Y_test = load_labels("data/t10k-labels-idx1-ubyte.gz")

In [9]:
print('Training data shapes: ')
print(X_train.shape)
print(Y_train_unencoded.shape)
print(Y_train.shape)
print('Test data shapes: ')
print(Y_test.shape)
print(X_test.shape)

Training data shapes: 
(60000, 784)
(60000, 1)
(60000, 10)
Test data shapes: 
(10000, 1)
(10000, 784)


# Coding Forward Propagation

Here is a reprint of `forward`, a core function of our perceptron:

```python
def forward(X, w):
  weighted_sum = np.matmul(X, w)
  return sigmoid(weighted_sum)
```

`forward()` implements the operation that we called “forward propagation”:
* It calculates the system’s outputs from the system’s inputs. 

In the case of the perceptron, it applies a weighted sum followed by a sigmoid. In the case of a neural network, things become slightly more complicated.

Let's start by defining our activations functions: sigmoid and softmax.

  $sigmoid(z)=\frac{1}{1+e^{-z}}$

  $softmax(l_i)=\frac{e^{l_i}}{∑ e^{l}}$


In [10]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [11]:
def softmax(logits):
  exponentials = np.exp(logits)
  denominator = np.sum(exponentials, axis=1).reshape(-1,1)
  return exponentials / denominator

Let's try the new softmax function. (We already tried the sigmoid function before)

In [13]:
a = np.array([[0.3, 0.8, 0.2],
             [0.1, 0.9, 0.1]])

In [14]:
softmax(a)

array([[0.28140804, 0.46396343, 0.25462853],
       [0.23665609, 0.52668782, 0.23665609]])

What happen if we want to use only one sample?

In [17]:
softmax(a[0])

AxisError: ignored

In [16]:
softmax(a[0].reshape(1, -1))

array([[0.28140804, 0.46396343, 0.25462853]])

What happen if our probabilities divierge a lot?

In [18]:
b = np.array([1,20]).reshape(1, -1)
c = np.array([1,1000]).reshape(1, -1)

In [19]:
softmax(b)

array([[5.60279641e-09, 9.99999994e-01]])

In [20]:
softmax(c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


array([[ 0., nan]])

Our implementations of `softmax()` have a problem: it is numerically unstable, meaning that they amplify small changes in the inputs.

In other woreds, `softmax()` function is prone to two issues: **overflow** and **underflow**

**Overflow**: It occurs when very large numbers are approximated as infinity

**Underflow**: It occurs when very small numbers (near zero in the number line) are approximated (i.e. rounded to) as zero

To combat these issues when doing softmax computation, a common trick is to shift the input vector by subtracting the maximum element in it from all elements. For the input vector *logits*, define *z* such that:

```python
z = logits - np.max(logits, axis=1).reshape(-1,1)
```
And then we take the softmax fo the new (stable) vector *z*.

In [21]:
def stable_softmax(logits):
    z = logits - np.max(logits, axis=1).reshape(-1,1)
    numerator = np.exp(z)
    denominator = np.sum(numerator,axis=1).reshape(-1,1)
    return numerator/denominator

Let's check that everything is still working correctly

In [22]:
stable_softmax(c)

array([[0., 1.]])

In [23]:
stable_softmax(a)

array([[0.28140804, 0.46396343, 0.25462853],
       [0.23665609, 0.52668782, 0.23665609]])

In [24]:
stable_softmax(b)

array([[5.60279641e-09, 9.99999994e-01]])

In [25]:
softmax(a)

array([[0.28140804, 0.46396343, 0.25462853],
       [0.23665609, 0.52668782, 0.23665609]])

In [26]:
softmax(b)

array([[5.60279641e-09, 9.99999994e-01]])

In [27]:
softmax(c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


array([[ 0., nan]])

Very well, after defining our activation functions, Let's move to implement the forward function.

The first step of forward propagation is the same as a regular perceptron’s:

In [52]:
def forward(X, w1, w2):
  h = sigmoid(np.matmul(prepend_bias(X), w1))
  y_hat = stable_softmax(np.matmul(prepend_bias(h), w2))
  return y_hat, h

N:ow we have the `forward()` function, and both activation functions—the sigmoid and the softmax. 

To complete the classifier, however, we still need to take care of the higher-level classification and testing function: `predict()` and `report()`.

Let's remember our previous implementations of predict function.

```python
def predict(X, beta, return_proba=False):
  y_hat =  forward(X,beta)
  if return_proba:
    return y_hat
  else:
    labels = np.argmax(y_hat, axis=1)
    return labels.reshape(-1,1)
```

In [57]:
def predict(X, w1, w2, return_proba=False):
  y_hat, _ =  forward(X, w1, w2)
  if return_proba:
    return y_hat
  else:
    labels = np.argmax(y_hat, axis=1)
    return labels.reshape(-1,1)

We are going to create a function called `report` for measuring and reporting the performance of our model (in terms of accuracy) during our training process. 

The function will give information during each step of our Gradient Descent process. It will do the following:

* Obtain the estimations $\hat{y}$ using the training data
* Compute the loss 
* Get the predictions (classifications)
* Compute the accuracy
* Report everything

In [65]:
def report(iteration, X_train, Y_train, X_test, Y_test, w1, w2):
  y_hat = forward(X_train, w1, w2)
  # y_hat = predict(X_train, w1, w2, return_proba=False)
  training_loss = loss(Y_train, y_hat)
  classifications = predict(X_test, w1, w2)
  accuracy = np.average(classifications==Y_test) * 100
  print(f'Iteration : {iteration} Loss : {training_loss:.6f} Accuracy : {accuracy:.2f}')

## A new loss function

So far, we used the log loss formula for our binary classifiers. We even used the log loss when we bundled ten binary classifiers in a multiclass classifier.

In that case, we added together the losses of the ten classifiers to get a total loss.

While the log loss served us well so far, it’s time to switch to a simpler formula
—one that’s specific to multiclass classifiers. It’s called the cross-entropy loss,
and it looks like this:

$L(y,\hat{y})=\frac{1}{n} \sum_{i=1}^{n} y^{(i)} \cdot log(\hat{y}^{(i)}) $

In [61]:
def loss(Y, y_hat):
  # print(Y.shape)
  # print(y_hat)
  print(y_hat.shape)
  return -np.sum(Y * np.log(y_hat)) / Y.shape[0]

There is a pragmatic reason to use the cross-entropy loss in our neural network: it’s a perfect match for the softmax. More specifically, a softmax followed by a cross-entropy loss makes it easier to code gradient descent.

With the 'loss()' function, half of our neural network’s logic is done—the half that takes care
of the `classification` phase.

This code, however, won’t be useful until we’ve also written the code that trains the network. 

Let's move to that!



# Backpropagation



After learning about backpropagation on the slides, let's implement that function. 

Let's start by definining the gradient of the sigmoid function

$\sigma^{'}=\sigma (1-\sigma)$

In [33]:
def sigmoid_gradient(sigmoid):
  return np.multiply(sigmoid, (1-sigmoid))

Now let's define the backpropagation function:

* $\frac{\partial{L}}{\partial{w_1}}=(\hat{y}-y)\cdot h$
* $\frac{\partial{L}}{\partial{w_2}}=(\hat{y}-y)\cdot w_2 \cdot \sigma \cdot (1-\sigma) \cdot x$

In [42]:
def back(X, Y, y_hat, w2, h):
  w2_gradient = np.matmul(prepend_bias(h).T, (y_hat - Y)) / X.shape[0]
  w1_gradient = np.matmul(prepend_bias(X).T, np.matmul((y_hat - Y), w2[1:].T)*sigmoid_gradient(h)) / X.shape[0]
  return (w1_gradient, w2_gradient)

## Las step, initializing our weights

As we just discussed. We should initialize our weights randomly.

How lare or how small those values should be?

Let's check something...

In [34]:
weighted_sum = 1000 * 10
sigmoid(weighted_sum)

1.0

In [35]:
sigmoid_gradient(sigmoid(weighted_sum))

0.0

Remember the chain rule? During backpropagation, this gradient of 0 gets
multiplied by the other local gradients, causing the entire gradient to become 0. With a gradient of 0, gradient descent has nowhere to go.

Let’s wrap up what we said in the previous sections. We should initialize
weights with values that are:
* Random (to break symmetry)
* Small (to speed up training and avoid dead neurons)


Let's use an empirical rule of thumb. We’ll make each weight range from 0 to
something around the following value, where $r$ is the number of rows in the weight matrix:

$w \approx \pm \sqrt{\frac{1}{r}} $

Let's implement our function:

In [36]:
def initialize_weights(n_input_variables, n_hidden_nodes, n_classes):
  w1_rows = n_input_variables + 1
  w1 = np.random.randn(w1_rows, n_hidden_nodes) * np.sqrt(1/w1_rows)
  w2_rows = n_hidden_nodes + 1
  w2 = np.random.randn(w2_rows, n_classes) * np.sqrt(1/w2_rows)
  return w1, w2

Remember that each matrix of weights
in a neural network has as many rows as its input elements, and as many
columns as its output elements.

Note: If you initialize a neural network with random weights, then you should expect a
slightly different loss every time you train it. To avoid that randomess, let's seed NumPy’s random number generator with `np.random.seed(a_known_seed)`.

In [41]:
np.random.seed(712)

Everything is finished now. Let's create our train function and let's try it!!!

In [39]:
def train(X_train, Y_train, X_test, Y_test, n_hidden_nodes, iterations, lr):
    n_input_variables = X_train.shape[1]
    n_classes = Y_train.shape[1]
    w1, w2 = initialize_weights(n_input_variables, n_hidden_nodes, n_classes)
    for iteration in range(iterations):
        y_hat, h = forward(X_train, w1, w2)
        w1_gradient, w2_gradient = back(X_train, Y_train, y_hat, w2, h)
        w1 = w1 - (w1_gradient * lr)
        w2 = w2 - (w2_gradient * lr)
        report(iteration, X_train, Y_train, X_test, Y_test, w1, w2)
    return w1, w2

In [64]:
%%time
w1, w2 = train(X_train, Y_train, X_test, Y_test, n_hidden_nodes=200, iterations = 20, lr=0.01)

AttributeError: ignored

## Summary 

Let’s recap what we learned today:

* In this lab session, you learned backpropagation—an algorithm to calculate the
gradients of the weights in a neural network.

* Those gradients represent the
impact of each weight on the overall loss.
Each training iteration in a neural network ping-pongs between two steps:
  1. **Forward propagation**: the network calculates each layer from the previous
one, from the input layer to the output ŷ.
  2. **Backpropagation**: the network bounces its way back from the output layer
to the weights, using the chain rule to calculate their gradients. Then it
descends those gradients, pushing the loss down, and $\hat{y}$ closer to the ground truth $y$.

* You also learned that neural networks don’t train well if all the weights have
the same value.
  * Instead, you should break their symmetry by initializing the weights with random values. 
  *Those initial values should also be small to avoid problems like overflows and dead neurons.